In [ ]:
!pip install split-folders

In [ ]:
import pandas as pd
import numpy as np
import keras
import glob
import matplotlib.pyplot as plt
import scipy
import seaborn as sns
from mlxtend.preprocessing import minmax_scaling
from sklearn.metrics import roc_curve, auc

from keras.utils.np_utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, GlobalAveragePooling2D, Input, BatchNormalization, Multiply, Activation

from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator

from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from keras import backend as K
# import splitfolders

import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import os

In [ ]:
# !pip install mlxtend
# !pip install utils

# Starting

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# splitfolders.ratio('/content/drive/MyDrive/capstone/resized_data', output="/content/drive/MyDrive/capstone", seed=1337, ratio=(.6, 0.2,0.2)) 

Copying files: 21874 files [19:17, 18.90 files/s]


In [ ]:
train_path="/content/drive/MyDrive/train"
#test_path="/content/drive/MyDrive/test"
val_path="/content/drive/MyDrive/val"

In [ ]:
food = ['bananas','bread','peas','rice','tomatoes']
x_train=[]

for folder in food:

    sub_path=train_path+"/"+folder

    for img in os.listdir(sub_path):

        image_path=sub_path+"/"+img

        img_arr=cv2.imread(image_path)        

        x_train.append(img_arr)


''' x_test=[]

for folder in os.listdir(test_path):

    sub_path=test_path+"/"+folder

    for img in os.listdir(sub_path):

        image_path=sub_path+"/"+img

        img_arr=cv2.imread(image_path)        

        x_test.append(img_arr) '''


x_val=[]

for folder in food:

    sub_path=val_path+"/"+folder

    for img in os.listdir(sub_path):

        image_path=sub_path+"/"+img

        img_arr=cv2.imread(image_path)       

        x_val.append(img_arr)

In [ ]:
train_x=np.array(x_train)
#test_x=np.array(x_test)
val_x=np.array(x_val)

In [ ]:
train_x=train_x/255.0
#test_x=test_x/255.0
val_x=val_x/255.0

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255)
#test_datagen = ImageDataGenerator(rescale = 1./255)
val_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:

training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (128, 128),
                                                 batch_size = 32,
                                                 class_mode = 'sparse')

'''test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (128, 128),
                                            batch_size = 32,
                                            class_mode = 'sparse')'''


val_set = val_datagen.flow_from_directory(val_path,
                                            target_size = (128, 128),
                                            batch_size = 32,
                                            class_mode = 'sparse')

Found 13121 images belonging to 5 classes.
Found 4372 images belonging to 5 classes.


In [ ]:
train_y=training_set.classes
''' test_y = test_set.classes '''
val_y = val_set.classes


In [ ]:
train_y.shape

(13121,)

In [ ]:
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (5,5), strides = 2, padding = 'Same', activation ='relu', input_shape = (128,128,3), kernel_initializer='he_normal'))
model.add(Conv2D(filters = 32, kernel_size = (5,5), strides = 2, padding = 'Same', activation ='relu',kernel_initializer='he_normal'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.4))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu',kernel_initializer='he_normal'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu',kernel_initializer='he_normal'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.4))
model.add(Conv2D(filters = 128, kernel_size = (2,2),padding = 'Same', activation ='relu',kernel_initializer='he_normal'))
model.add(Conv2D(filters = 128, kernel_size = (2,2),padding = 'Same', activation ='relu',kernel_initializer='he_normal'))
model.add(GlobalAveragePooling2D())
model.add(Dense(64, activation = "relu",kernel_initializer='he_normal'))
model.add(Dropout(0.4))
model.add(Dense(512, activation = "relu",kernel_initializer='he_normal'))
model.add(Dropout(0.4))

model.add(Dense(5, activation = "softmax",kernel_initializer='he_normal',kernel_regularizer=l2()))

#callbacks
'''checkpointer = ModelCheckpoint(filepath='model.hdf5', verbose=1, save_best_only=True, save_weights_only=True)
earlystopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=20, mode='auto') '''


model.compile(optimizer = 'Adam' , loss = "sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
del model


In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 64, 64, 32)        2432      
                                                                 
 conv2d_7 (Conv2D)           (None, 32, 32, 32)        25632     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 16, 16, 32)       0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 16, 16, 32)        0         
                                                                 
 conv2d_8 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 conv2d_9 (Conv2D)           (None, 16, 16, 64)        36928     
                                                      

In [ ]:
train_y

array([0, 0, 0, ..., 4, 4, 4], dtype=int32)

In [ ]:
history = model.fit_generator(training_set,steps_per_epoch=2250/64,
                              validation_data=val_set,validation_steps=750/64, 
                              epochs=30)
'''model.fit(train_x, train_y, batch_size=64, epochs=30, validation_data=(val_x,val_y.flatten()))'''


In [ ]:
# serialize model to JSON
model_json = model.to_json()


In [ ]:
with open("256_512_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("256_256_model.h5")
print("Saved model to disk")

Saved model to disk


In [ ]:
history

In [ ]:
# convert the history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(history.history) 

# save to json:  
hist_json_file = '256_512_history.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

# or save to csv: 
hist_csv_file = '256_256_history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

In [ ]:
hist_df

,loss,accuracy,val_loss,val_accuracy
0,1.677049,0.275174,1.439848,0.382812
1,1.363843,0.427951,1.265693,0.481771
2,1.252312,0.515625,1.081932,0.598958
3,1.128416,0.543403,1.158349,0.463542
4,1.099479,0.547743,1.067525,0.591146
5,1.009112,0.584201,1.109130,0.617188
6,0.989819,0.630208,0.960704,0.606771
7,0.955797,0.614583,0.960372,0.627604
8,0.946111,0.636285,1.179101,0.528646
9,0.907585,0.666667,1.100601,0.565104


### Loading a saved model

In [ ]:
import tensorflow as tf

# load json and create model
json_file = open('model_v1.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model =  tf.keras.models.model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='sparse_categorical_cross_entropy', optimizer='Adam', metrics=['accuracy'])


Loaded model from disk


'score = loaded_model.evaluate(test_x, test_y, verbose=0)\nprint("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))'

In [ ]:
loaded_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 64, 64, 32)        2432      
                                                                 
 conv2d_13 (Conv2D)          (None, 32, 32, 32)        25632     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 16, 16, 32)       0         
 2D)                                                             
                                                                 
 dropout_6 (Dropout)         (None, 16, 16, 32)        0         
                                                                 
 conv2d_14 (Conv2D)          (None, 16, 16, 64)        18496     
                                                                 
 conv2d_15 (Conv2D)          (None, 16, 16, 64)        36928     
                                                      

In [ ]:
test_img = cv2.imread(img_test_path)
test_img = cv2.resize(test_img,(128,128))
test_img = np.array(test_img)
test_img = test_img/255.0

test_arr = np.array( [test_img])
test_arr.shape

In [ ]:

np.argmax(model.predict(test_arr),axis=1)


In [ ]:
import gc 
del train_x
del val_x
del train_datagen
del val_datagen
gc.collect()

636

In [ ]:
test_path="/content/drive/MyDrive/test"

x_test=[]

for folder in os.listdir(test_path):

    sub_path=test_path+"/"+folder

    for img in os.listdir(sub_path):

        image_path=sub_path+"/"+img

        img_arr=cv2.imread(image_path)

        x_test.append(img_arr)

  
test_x=np.array(x_test)
test_x=test_x/255.0
test_datagen = ImageDataGenerator(rescale = 1./255)

test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (128, 128),
                                            batch_size = 32,
                                            class_mode = 'sparse')

test_y = test_set.classes

Found 4381 images belonging to 5 classes.


In [ ]:
y_pred = np.argmax(loaded_model.predict(test_x), axis = 1)

In [ ]:
accuracy_score(test_y, y_pred)
confusion_matrix(test_y, y_pred)

array([[ 19, 808,   3,  27,  18],
       [  5,  46,  17, 827,  13],
       [  2,   2,  11,  14, 821],
       [ 17,   1, 826,   9,  33],
       [832,   8,   8,  13,   1]])

In [ ]:

score = loaded_model.evaluate(test_x, test_y, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100)) 






In [ ]:
#predict
y_pred=model.predict(test_x)

In [ ]:
model.evaluate(test_x,test_y,batch_size=32)